# Cleaning Melbourne sensor footfall data 
This script cleans the footfall data for Melbourne

#### Code initialisation

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
from datetime import date, datetime

from tests.conftest import data_dir

# Add the path two directories up from the current notebook location
sys.path.append(os.path.abspath(os.path.join('..')))
# Now import the function from DataPrepFunctions.py
from DataPrepFunctions import check_and_prepare_data_directory

#### Import pedestrian count data

TODO: Need to upload the data to a permanent location and check that the download/extract function works.

In [18]:
check_and_prepare_data_directory(data_dir="../../Data/", 
                                 file_url="www.my_domain/my_data.zip",)
sensor_counts = pd.read_csv('../../Data/FootfallData/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv')
sensor_locations = pd.read_csv('../../Data/FootfallData/melbourne_locations.csv')

11
Data directory '../../Data/' already contains files.


#### Rename columns to all lowercase (to facilitate joining)

In [19]:
sensor_counts.rename({'Date_Time': 'datetime', 'Year': 'year', 'Month':'month', 'Mdate': 'mdate', 
                      'Day': 'day', 'Time': 'time', 'Sensor_ID': 'sensor_id', 'Hourly_Counts': 'hourly_counts'}, 
                     axis = 1, inplace = True)

#### Drop unneeded columns

In [20]:
sensor_counts.drop(['ID', 'Sensor_Name'], axis = 1, inplace = True)
#sensor_locations.drop(['sensor_description', 'sensor_name', 'installation_date', 'status', 'note', 'direction_1',
#                      'direction_2'], axis = 1, inplace = True)

#### Join two dataframes so location and count info in same place

In [21]:
location_counts = pd.merge(sensor_locations, sensor_counts, on='sensor_id', how='inner')

#### It could be useful to drop sensors that do not have as many recorded countsfor now leaving them in, but may drop in the future

#### Properly format datetime column

In [22]:
location_counts['datetime'] = pd.to_datetime(location_counts['datetime'], format = '%B %d, %Y %I:%M:%S %p')

#### Order by datetime column

In [23]:
location_counts = location_counts.sort_values(by=['datetime'])
location_counts.reset_index(inplace = True, drop = True)

#### September, 2010 dates have problems:
All dates have a timestamp of 00:00, presume they are in order of hour of day, for each day there are only 23 hours worth of data
For now, in later stages just filter out 2010 data

In [24]:
location_counts = location_counts[location_counts['year']>2010]

### Clean and save data

In [25]:
# location_counts.drop(['Latitude', 'Longitude', 'location'], axis = 1, inplace =True)
# location_counts.rename({'day': 'weekday', 'mdate': 'day', 'time': 'hour'}, axis =1, inplace = True)

# Change month to integers 1-12
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June':6, 'July': 7, 'August': 8,
         'September': 9, 'October': 10, 'November': 11, 'December': 12}

location_counts.month = location_counts.month.map(months)

In [26]:
just_some_sensors = location_counts[location_counts['sensor_id'].isin([2,6,8,9,10,11,18])]
just_some_sensors.to_csv("../../Cleaned_data/validsensors.csv",index=False)
location_counts.to_csv("../../Cleaned_data/allsensors.csv",index=False)

In [11]:
# # Get data for just one sensor
# one_sensor =  location_counts[location_counts.sensor_id == 4]
# one_year = one_sensor[one_sensor.year==2010]
# # Set the datetime as the index
# one_sensor.set_index('datetime', inplace = True)
# duplicates = one_sensor[one_sensor.index.duplicated()]
# for day in duplicates.mdate.unique():
#     one_day = duplicates[duplicates.mdate == day]
#     print(day, len(one_day))

## Outlier removal

(_I think this is redundant now, performed before modelling_)

TODO: Remove following this redundant outlier removal code?

In [ ]:
def doubleMADsfromMedian(y,thresh=3.5):
    """Find outliers using the Median Average Distance.
    
    VALUE: return a list of true/false denoting whether the element in y is an outlier or not
    
    PARAMETERS:
      - y is a pandas Series, or something like that.
    
    warning: this function does not check for NAs
    nor does it address issues when 
    more than 50% of your data have identical values
    """
    # Calculate the upper and lower limits
    m = np.median(y) # The median
    abs_dev = np.abs(y - m) # The absolute difference between each y and the median
    # The upper and lower limits are the median of the difference
    # of each data point from the median of the data
    left_mad = np.median(abs_dev[y <= m]) # The left limit (median of lower half)
    right_mad = np.median(abs_dev[y >= m]) # The right limit (median of upper half)
    
    # Now create an array where each value has left_mad if it is in the lower half of the data,
    # or right_mad if it is in the upper half
    y_mad = left_mad * np.ones(len(y)) # Initially every value is 'left_mad'
    y_mad[y > m] = right_mad # Now larger values are right_mad

    # Calculate the z scores for each element
    modified_z_score = 0.6745 * abs_dev / y_mad
    modified_z_score[y == m] = 0
    
    # Return boolean list showing whether each y is an outlier
    return modified_z_score > thresh

# Make a list of true/false for whether the footfall is an outlier
no_outliers = pd.DataFrame(doubleMADsfromMedian(original['InCount']))
no_outliers.columns = ['outlier'] # Rename the column to 'outlier'

# Join to the original footfall data to the list of outliers, then select a few useful columns
join = pd.concat([original, no_outliers], axis = 1)
join = pd.DataFrame(join, columns = ['Day_yr', 'outlier', 'InCount'])

# Choose just the outliers
outliers = join[join['outlier'] == True]
outliers_list = list(outliers['Day_yr']) # A list of the days that are outliers

# Now remove all outliers from the original data
df = original.loc[~original['Day_yr'].isin(outliers_list)]
df = df.reset_index(drop = True)

# Check that the lengths all make sense
assert(len(df) == len(original)-len(outliers_list))

print("I found {} outliers from {} days in total. Removing them leaves us with {} events".format(\
    len(outliers_list), len(join), len(df) ) )